In [2]:
! pip install transformers

     |████████████████████████████████| 1.3 MB 841 kB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 666 kB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 9.0 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=7cb106125d0f3a3566fc1a723adf53e26e240989c7326884309b663618d860c4
  Stored in directory: /home/jovyan/.cache/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

BERT_MODEL='bert-base-cased'
# BERT_MODEL='dmis-lab/biobert-base-cased-v1.1'

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False)
model = BertModel.from_pretrained(BERT_MODEL, output_hidden_states = True)

In [ ]:
def get_hidden_states(model,tokenizer,marked_text):
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]
        return hidden_states

In [ ]:
def get_embeddings(text):
    marked_text = "[CLS] " + text + " [SEP]"
    
    hidden_states = get_hidden_states(model, tokenizer, marked_text)
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    
    return token_embeddings

In [ ]:
input_sentence = "A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes."
tokenized_input = tokenizer.tokenize(input_sentence)

In [ ]:
first_embeddings = []
last_embeddings = []

with open("embeddings/label.tsv", "w") as labels_file:
    for token in tokenized_input:
        if (not token.startswith("##")):
            marked_text = "[CLS] " + token + " [SEP]"

            print(token, file=labels_file)

            token_embeddings = get_embeddings(marked_text)
        
            first_embedding_layer = token_embeddings[1][1]
            last_embedding_layer = token_embeddings[1][12]

            tsv_rows = ''
            for e in last_embedding_layer:
                tsv_rows += str(e.item()) + '\t'
            last_embeddings.append(tsv_rows)
            
            tsv_rows = ''
            for e in first_embedding_layer:
                tsv_rows += str(e.item()) + '\t'
            first_embeddings.append(tsv_rows)

In [ ]:
with open("bert/first_embedding.tsv", "w") as f:
    for e in first_embeddings:
        print (e, file=f)
        
with open("bert/last_embedding.tsv", "w") as f:
    for e in last_embeddings:
        print (e, file=f)